In [48]:
#importing library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
#LinearSVR=Linear based kernel SVR=Non Linear based kernel
from sklearn.svm import LinearSVR,SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings(action='ignore')

In [49]:
#loading the dataset
df=pd.read_csv('../input/vegetable-market/Vegetable_market.csv')
df

,Vegetable,Season,Month,Temp,Deasaster Happen in last 3month,Vegetable condition,Price per kg
0,potato,winter,jan,15,no,fresh,20
1,tomato,winter,jan,15,no,fresh,50
2,peas,winter,jan,15,no,fresh,70
3,pumkin,winter,jan,15,no,fresh,25
4,cucumber,winter,jan,15,no,fresh,20
...,...,...,...,...,...,...,...
116,brinjal,winter,jan,15,yes,fresh,33
117,ginger,winter,jan,15,no,fresh,88
118,potato,summer,apr,32,no,fresh,24
119,peas,summer,apr,33,no,fresh,33


In [50]:
#onehot_encode
def onehot_encode(df,column):
    df=df.copy()
    dummies=pd.get_dummies(df[column])
    df=pd.concat([df,dummies],axis=1)
    df=df.drop(column,axis=1)
    return df

In [51]:
#Preprocessing 
def preprocess_inputs(df):
    df=df.copy()
    #clearnign vegetable condition
    df['Vegetable condition']=df['Vegetable condition'].replace({'scarp':'scrap'})
    #replacing yes with 1 and no with 0 with replace method
    df['Deasaster Happen in last 3month']=df['Deasaster Happen in last 3month'].replace({'yes':1,'no':0})
    df['Month']=df['Month'].replace({'jan':1,
  'apr':4,
  'july':7,
  'sept':9,
  'oct':10,
  'dec':12,
  'may':5,
  'aug':8,
  'june':6,
  ' ':np.NaN})
    df['Month']=df['Month'].fillna(df['Month'].mode()[0])
    for column in ['Vegetable', 'Season', 'Month', 'Vegetable condition']:
        df=onehot_encode(df,column)
    #split the data between feature and target dataset
    y=df['Price per kg']
    x=df.drop('Price per kg',axis=1)
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,shuffle=True,random_state=1)
    #scaling the dataset
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.fit_transform(x_train),columns=x_train.columns,index=x_train.index)
    x_test=pd.DataFrame(scaler.fit_transform(x_test),columns=x_test.columns,index=x_test.index)
    return x_train,x_test,y_train,y_test

In [52]:
x_train,x_test,y_train,y_test=preprocess_inputs(df)
x_train

,Temp,Deasaster Happen in last 3month,Bitter gourd,Raddish,brinjal,cabage,califlower,chilly,cucumber,garlic,...,6,7,8,9,10,12,march,avarage,fresh,scrap
80,0.135584,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,2.236068,-1.452966,-0.427900
38,0.569452,1.628550,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,3.316625,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,-0.447214,0.688247,-0.427900
19,0.894854,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,-0.447214,0.688247,-0.427900
120,0.786387,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,-0.447214,0.688247,-0.427900
27,1.437189,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,-0.447214,0.688247,-0.427900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,-1.057554,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,-0.447214,0.688247,-0.427900
72,-1.057554,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,3.974921,-0.19245,-0.447214,-1.452966,2.336993
12,-1.057554,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,3.316625,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,-0.251577,-0.19245,-0.447214,0.688247,-0.427900
107,-0.406752,-0.614043,3.974921,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,-0.19245,...,-0.156174,-0.301511,-0.192450,-0.109764,-0.109764,3.974921,-0.19245,-0.447214,-1.452966,2.336993


In [53]:
#mapping  unique categorical values with respect to each column
{column:len(x[column].unique()) for column in x.columns}

{'Temp': 17,
 'Deasaster Happen in last 3month': 2,
 'Price per kg': 43,
 'Bitter gourd': 2,
 'Raddish': 2,
 'brinjal': 2,
 'cabage': 2,
 'califlower': 2,
 'chilly': 2,
 'cucumber': 2,
 'garlic': 2,
 'ginger': 2,
 'okra': 2,
 'onion': 2,
 'peas': 2,
 'pointed grourd ': 2,
 'potato': 2,
 'pumkin': 2,
 'radish': 2,
 'tomato ': 2,
 'autumn': 2,
 'monsoon': 2,
 'spring': 2,
 'summer': 2,
 'winter': 2,
 1: 2,
 4: 2,
 5: 2,
 6: 2,
 7: 2,
 8: 2,
 9: 2,
 10: 2,
 12: 2,
 'march': 2,
 'avarage': 2,
 'fresh': 2,
 'scrap': 2}

In [54]:
#getting unique values for categorcal features in a dictionary
{column:list(x[column].unique()) for column in x.columns if x[column].dtypes=='object'}

{}

In [55]:
df['Month'].unique()

array(['jan', 'apr', 'july', 'sept', 'oct', 'dec', 'may', 'aug', 'june',
       ' ', 'march'], dtype=object)

In [56]:
#checking for missing value
x.isna().sum()

Temp                               0
Deasaster Happen in last 3month    0
Price per kg                       0
Bitter gourd                       0
Raddish                            0
brinjal                            0
cabage                             0
califlower                         0
chilly                             0
cucumber                           0
garlic                             0
ginger                             0
okra                               0
onion                              0
peas                               0
pointed grourd                     0
potato                             0
pumkin                             0
radish                             0
tomato                             0
autumn                             0
monsoon                            0
spring                             0
summer                             0
winter                             0
1                                  0
4                                  0
5

In [57]:
df['Month'].mode()

0    jan
dtype: object

In [58]:
[column for column in x.columns if x[column].dtypes=='object']

[]

In [60]:

models={'LinearRegressor':LinearRegression(),
'Ridge':Ridge(),
'Lasso':Lasso(),
'KNeightborsRegressor':KNeighborsRegressor(),
'MLPRegressor':MLPRegressor(),
'LinearSVR':LinearSVR(),
'SVR':SVR(),
'Decision Tree':DecisionTreeRegressor(),
'Random Forest Regressor':RandomForestRegressor(),
'Gradient Boosting Regressor':GradientBoostingRegressor(),
'XGB Regressor':XGBRegressor(),
'LGBM Regressor':LGBMRegressor(),
'Cat Boost Regressor':CatBoostRegressor(verbose=0)}

In [62]:
#training the model
for name,model in models.items():
    model.fit(x_train,y_train)
    print(name+'trained')

LinearRegressortrained
Ridgetrained
Lassotrained
KNeightborsRegressortrained
MLPRegressortrained
LinearSVRtrained
SVRtrained
Decision Treetrained
Random Forest Regressortrained
Gradient Boosting Regressortrained
XGB Regressortrained
LGBM Regressortrained
Cat Boost Regressortrained


In [64]:
#Results
for name,model in models.items():
    print(name+'R^2 Score:{:.5f}'.format(model.score(x_test,y_test)))

LinearRegressorR^2 Score:0.54596
RidgeR^2 Score:0.57560
LassoR^2 Score:0.63326
KNeightborsRegressorR^2 Score:0.27965
MLPRegressorR^2 Score:-0.31033
LinearSVRR^2 Score:0.32049
SVRR^2 Score:-0.13637
Decision TreeR^2 Score:0.56663
Random Forest RegressorR^2 Score:0.61491
Gradient Boosting RegressorR^2 Score:0.64496
XGB RegressorR^2 Score:0.64674
LGBM RegressorR^2 Score:0.17046
Cat Boost RegressorR^2 Score:0.65237
